Common PSO Variations:
Standard PSO 
PSO Binaire Amélioré 
Adaptive PSO (APSO) 



## **Problème de couverture (Covering Problem)**  
L'objectif est de choisir un ensemble de sous-ensembles pour couvrir un maximum d'éléments uniques.

U = \{ 1, 2, 3, 4, 5, 6 \}

Et plusieurs sous-ensembles disponibles :  

S_1 = \{ 1, 2, 3 \}, \quad S_2 = \{ 3, 4 \}, \quad S_3 = \{ 4, 5, 6 \}, \quad S_4 = \{ 2, 5 \}

Notre objectif est de sélectionner certains de ces sous-ensembles pour couvrir **le plus grand nombre d'éléments possibles** de \( U \).



## **1. Représentation d’une solution (Encodage binaire)**  
On utilise un **vecteur binaire** où **chaque bit indique si un sous-ensemble est sélectionné ou non**.


x = (x_1, x_2, x_3, x_4)


 \( x_1 = 1 \) signifie qu'on sélectionne \( S_1 \).  


Par exemple, si on choisit **\( x = (1, 0, 1, 0) \)**, cela signifie que nous avons sélectionné \( S_1 \) et \( S_3 \), donc nous couvrons les éléments :

S_1 \cup S_3 = \{ 1, 2, 3 \} \cup \{ 4, 5, 6 \} = \{ 1, 2, 3, 4, 5, 6 \}

Nous avons **couvert tous les éléments** de \( U \), donc c'est une bonne solution.


## **2. Taille de l’espace des solutions**  
Si on choisit exactement **\( k \)** sous-ensembles parmi **\( m \)** disponibles, alors le nombre total de solutions possibles est donné par :

6


Donc, il y a **6 façons différentes de choisir 2 sous-ensembles parmi 4**.

Si **on peut choisir n'importe quel nombre de sous-ensembles**, alors **chaque sous-ensemble peut être inclus ou non**, donc le nombre total de solutions est :


2^m


## **3. Fonction de fitness (objectif)**  
On veut maximiser le nombre d’éléments couverts. Une **fonction de fitness** mesure **combien d'éléments uniques sont couverts** par une solution donnée.


## **4. Ajout de pénalités (si nécessaire)**  
Si on impose une **contrainte** (par exemple, sélectionner au maximum \( k \) sous-ensembles), on ajoute une pénalité :

1. **Représentation** : Vecteur binaire (\( 1 \) = sous-ensemble sélectionné, \( 0 \) = non sélectionné).  
2. **Espace des solutions** : \( \binom{m}{k} \) si on choisit exactement \( k \) sous-ensembles, sinon \( 2^m \).  
3. **Fitness** : Nombre d'éléments uniques couverts.  
4. **Pénalité** : Ajoutée si des contraintes (ex. limite sur \( k \)) sont imposées.



In [39]:
import os

def load_set_cover_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    N, M = map(int, lines[0].split())
    universe = set(range(1, N + 1))  # Ensemble universel avec indices à partir de 1
    subsets = [set() for _ in range(M)]
    
    for i in range(1, len(lines)):
        line = list(map(int, lines[i].split()))
        if not line:
            continue
        
        count = line[0]  # Nombre de sous-ensembles contenant cet élément
        subset_indices = line[1:count+1]  # Liste des sous-ensembles de cet élément
        element = i  # Indice de l'élément commence à 1
        
        for s in subset_indices:
            if 0 <= s < M:
                subsets[s].add(element)
    
    return universe, subsets





In [50]:
file_path41 = os.path.join("Preblem_set_4", "scp41.txt")
file_path42 = os.path.join("Preblem_set_4", "scp42.txt")
file_path43 = os.path.join("Preblem_set_4", "scp43.txt")
file_path44 = os.path.join("Preblem_set_4", "scp44.txt")
file_path45 = os.path.join("Preblem_set_4", "scp45.txt")
universe41, subsets41 = load_set_cover_from_file(file_path41)
universe42, subsets42 = load_set_cover_from_file(file_path42)
universe43, subsets43 = load_set_cover_from_file(file_path43)
universe44, subsets44 = load_set_cover_from_file(file_path44)
universe45, subsets45 = load_set_cover_from_file(file_path45)

In [51]:
#pso standard
import numpy as np
import random

def binary_to_decimal(binary_vector):
    """Convertit un vecteur binaire en nombre décimal."""
    return int("".join(map(str, binary_vector)), 2)

def decimal_to_binary(decimal_number, size):
    """Convertit un nombre décimal en vecteur binaire de taille fixée."""
    return [int(x) for x in format(decimal_number, f'0{size}b')]

def fitness_function(binary_vector, subsets, universe):
    """Calcule la couverture des ensembles sélectionnés."""
    covered_elements = set()
    for i, selected in enumerate(binary_vector):
        if selected:
            covered_elements.update(subsets[i])
    return len(covered_elements)

def enforce_k_selection(binary_vector, k, subsets, universe):
    """Assure que seulement k sous-ensembles sont sélectionnés en désactivant les moins utiles."""
    selected_indices = [i for i, val in enumerate(binary_vector) if val == 1]
    if len(selected_indices) > k:
        # Calculer la contribution de chaque sous-ensemble
        contributions = {}
        total_covered = set()
        for idx in selected_indices:
            total_covered.update(subsets[idx])
        
        for idx in selected_indices:
            unique_coverage = subsets[idx] - (total_covered - subsets[idx])
            contributions[idx] = len(unique_coverage)
        
        # Trier les indices en fonction de leur contribution croissante
        sorted_indices = sorted(selected_indices, key=lambda i: contributions[i])
        
        # Désactiver les moins utiles
        for i in sorted_indices[:len(selected_indices) - k]:
            binary_vector[i] = 0
    elif len(selected_indices) < k:
        available_indices = [i for i, val in enumerate(binary_vector) if val == 0]
        random.shuffle(available_indices)
        for i in available_indices[:k - len(selected_indices)]:
            binary_vector[i] = 1
    return binary_vector

# Paramètres
num_particles = 30
num_dimensions = 1000  # Nombre de sous-ensembles disponibles
num_iterations = 100
w = 0.9
c1, c2 = 1, 1  # Coefficients d'accélération
k = int(1000/25)  # Nombre de sous-ensembles à sélectionner



def particle_swarm_optimization_standard(subsets, universe, num_particles=30, num_iterations=100, k=k, c1=1.5, c2=1.5, w=0.7):
    num_dimensions = len(subsets)
    particles = []
    velocities = [random.uniform(-1, 1) for _ in range(num_particles)]
    
    for _ in range(num_particles):
        binary_vector = [0] * num_dimensions
        selected_indices = random.sample(range(num_dimensions), k)
        for idx in selected_indices:
            binary_vector[idx] = 1
        particles.append(binary_vector)
    
    personal_best = particles.copy()
    global_best = max(particles, key=lambda p: fitness_function(p, subsets, universe))
    
    for _ in range(num_iterations):
        for i in range(num_particles):
            velocities[i] = int(w * velocities[i] +
                                c1 * random.random() * (binary_to_decimal(personal_best[i]) - binary_to_decimal(particles[i])) +
                                c2 * random.random() * (binary_to_decimal(global_best) - binary_to_decimal(particles[i])))
            
            new_position = enforce_k_selection(particles[i], k, subsets, universe)
            
            if fitness_function(new_position, subsets, universe) > fitness_function(personal_best[i], subsets, universe):
                personal_best[i] = new_position
            
            particles[i] = new_position
        
        best_particle = max(particles, key=lambda p: fitness_function(p, subsets, universe))
        if fitness_function(best_particle, subsets, universe) > fitness_function(global_best, subsets, universe):
            global_best = best_particle
    
    covered_elements = set()
    for i, included in enumerate(global_best):
        if included:
            covered_elements.update(subsets[i])
    
    return global_best, len(covered_elements)
# Résultat final
global_best41,best_score41=particle_swarm_optimization_standard(subsets41, universe41, num_particles, num_iterations, k, c1, c2,w)
global_best42,best_score42=particle_swarm_optimization_standard(subsets42, universe42, num_particles, num_iterations, k, c1, c2,w )
global_best43,best_score43=particle_swarm_optimization_standard(subsets43, universe43, num_particles, num_iterations, k, c1, c2,w)
global_best44,best_score44=particle_swarm_optimization_standard(subsets44, universe44, num_particles, num_iterations, k, c1, c2,w)
global_best45,best_score45=particle_swarm_optimization_standard(subsets45, universe45, num_particles, num_iterations, k, c1, c2,w)
# --- Résultat final ---
print("Meilleure solution trouvée (scp41.tx):", global_best41)
print("Nombre d'éléments couverts (scp41.txt):", best_score41)

print("Meilleure solution trouvée (scp42.tx):", global_best42)
print("Nombre d'éléments couverts (scp42.txt):", best_score42)

print("Meilleure solution trouvée (scp43.tx):", global_best43)
print("Nombre d'éléments couverts (scp43.txt):", best_score43)

print("Meilleure solution trouvée (scp44.tx):", global_best44)
print("Nombre d'éléments couverts (scp44.txt):", best_score44)

print("Meilleure solution trouvée (scp45.tx):", global_best45)
print("Nombre d'éléments couverts (scp45.txt):", best_score45)

Meilleure solution trouvée (scp41.tx): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Mise à jour des vitesses incorrecte	La vitesse est un nombre entier, pas une proba de flip de bit	Utiliser une fonction sigmoïde (BPSO)

Sélection 𝑘 k sous-ensembles naïve	Désactive des sous-ensembles au hasard	Sélectionner intelligemment ceux qui couvrent le plus

Pas de critère d'arrêt	Continue 100 itérations même si la solution ne change plus	Arrêter après X itérations sans amélioration

PSO Binaire Amélioré 

In [52]:

import numpy as np
import random

# --- Fonctions d'encodage/décodage ---
def binary_to_decimal(binary_vector):
    return int("".join(map(str, binary_vector)), 2)

def decimal_to_binary(decimal_number, size):
    return [int(x) for x in format(decimal_number, f'0{size}b')]

# --- Fonction de fitness ---
def fitness_function(binary_vector, subsets, universe):
    covered_elements = set()
    for i, selected in enumerate(binary_vector):
        if selected:
            covered_elements.update(subsets[i])
    return len(covered_elements)

# --- Fonction sigmoïde pour PSO binaire ---
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# --- Sélection intelligente des sous-ensembles ---
def smart_selection(binary_vector, k, subsets, universe):
    selected = [i for i, bit in enumerate(binary_vector) if bit == 1]
    
    if len(selected) > k:
        selected = sorted(selected, key=lambda idx: len(subsets[idx]), reverse=True)[:k]
    elif len(selected) < k:
        remaining = [i for i in range(len(subsets)) if i not in selected]
        remaining = sorted(remaining, key=lambda idx: len(subsets[idx] - set.union(*[subsets[i] for i in selected])), reverse=True)
        selected += remaining[:k - len(selected)]
    
    return [1 if i in selected else 0 for i in range(len(subsets))]

# --- Paramètres PSO ---
num_particles = 50
num_dimensions = 1000  
num_iterations = 100
c1, c2 = 2.05, 2.05  # Valeurs classiques pour CF-PSO
phi = c1 + c2

# Calcul du facteur de constriction
chi = 2 / abs(2 - phi - np.sqrt(phi**2 - 4 * phi))

k = int(1000/20)

# Charger un seul fichier



# --- Définition du problème ---


def particle_swarm_optimization(subsets, universe, num_particles=30, num_iterations=100, k=5, c1=1.5, c2=1.5, chi=0.7):
    num_dimensions = len(subsets)
    particles = []
    velocities = []
    
    for _ in range(num_particles):
        binary_vector = [0] * num_dimensions
        selected_indices = random.sample(range(num_dimensions), k)
        for idx in selected_indices:
            binary_vector[idx] = 1
        particles.append(binary_vector)
        velocities.append([random.uniform(-1, 1) for _ in range(num_dimensions)])
    
    personal_best = particles.copy()
    global_best = max(particles, key=lambda p: fitness_function(p, subsets, universe))
    best_score = fitness_function(global_best, subsets, universe)
    no_improvement = 0
    
    for iteration in range(num_iterations):
        for i in range(num_particles):
            for d in range(num_dimensions):
                r1, r2 = random.random(), random.random()
                velocities[i][d] = (chi * (velocities[i][d] +
                                    c1 * r1 * (personal_best[i][d] - particles[i][d]) +
                                    c2 * r2 * (global_best[d] - particles[i][d])))
            binary_vector = [1 if random.random() < sigmoid(v) else bit for v, bit in zip(velocities[i], particles[i])]
            binary_vector = smart_selection(binary_vector, k, subsets, universe)
            
            if fitness_function(binary_vector, subsets, universe) > fitness_function(personal_best[i], subsets, universe):
                personal_best[i] = binary_vector
            particles[i] = binary_vector
        
        best_particle = max(particles, key=lambda p: fitness_function(p, subsets, universe))
        new_best_score = fitness_function(best_particle, subsets, universe)
        
        if new_best_score > best_score:
            best_score = new_best_score
            global_best = best_particle
            no_improvement = 0
        else:
            no_improvement += 1
        
        if no_improvement >= 10:
            break
    
    return global_best, best_score

global_best41,best_score41=particle_swarm_optimization(subsets41, universe41, num_particles, num_iterations, k, c1, c2, chi)
global_best42,best_score42=particle_swarm_optimization(subsets42, universe42, num_particles, num_iterations, k, c1, c2, chi)
global_best43,best_score43=particle_swarm_optimization(subsets43, universe43, num_particles, num_iterations, k, c1, c2, chi)
global_best44,best_score44=particle_swarm_optimization(subsets44, universe44, num_particles, num_iterations, k, c1, c2, chi)
global_best45,best_score45=particle_swarm_optimization(subsets45, universe45, num_particles, num_iterations, k, c1, c2, chi)
# --- Résultat final ---
print("Meilleure solution trouvée (scp41.tx):", global_best41)
print("Nombre d'éléments couverts (scp41.txt):", best_score41)

print("Meilleure solution trouvée (scp42.tx):", global_best42)
print("Nombre d'éléments couverts (scp42.txt):", best_score42)

print("Meilleure solution trouvée (scp43.tx):", global_best43)
print("Nombre d'éléments couverts (scp43.txt):", best_score43)

print("Meilleure solution trouvée (scp44.tx):", global_best44)
print("Nombre d'éléments couverts (scp44.txt):", best_score44)

print("Meilleure solution trouvée (scp45.tx):", global_best45)
print("Nombre d'éléments couverts (scp45.txt):", best_score45)

Meilleure solution trouvée (scp41.tx): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

Adaptive PSO (APSO)

In [4]:
import numpy as np

class APSO:
    def __init__(self, func, dim, bounds, num_particles=30, max_iter=100):
        self.func = func
        self.dim = dim
        self.bounds = np.array(bounds)
        self.num_particles = num_particles
        self.max_iter = max_iter
        
        # Paramètres adaptatifs
        self.w = 0.9  # Inertie initiale
        self.c1 = 2.5  # Coefficient cognitif
        self.c2 = 0.5  # Coefficient social
        
        # Initialisation
        self.pos = np.random.uniform(self.bounds[:, 0], self.bounds[:, 1], (num_particles, dim))
        self.vel = np.zeros((num_particles, dim))
        self.pbest = self.pos.copy()
        self.pbest_val = np.array([self.func(p) for p in self.pbest])
        self.gbest = self.pbest[np.argmin(self.pbest_val)]
        self.gbest_val = min(self.pbest_val)
    
    def update_parameters(self, iter):
        """ Mise à jour dynamique des paramètres """
        self.w = 0.9 - (0.5 * iter / self.max_iter)  # Réduction linéaire
        self.c1 = 2.5 - (2.0 * iter / self.max_iter)  # Diminue avec le temps
        self.c2 = 0.5 + (2.0 * iter / self.max_iter)  # Augmente avec le temps
    
    def mutate(self, particle_idx):
        """ Mutation aléatoire pour éviter la stagnation """
        mutation_prob = 0.1  # Probabilité de mutation
        if np.random.rand() < mutation_prob:
            dim_to_mutate = np.random.randint(0, self.dim)
            self.pos[particle_idx, dim_to_mutate] = np.random.uniform(self.bounds[dim_to_mutate, 0], self.bounds[dim_to_mutate, 1])
    
    def optimize(self):
        for iter in range(self.max_iter):
            self.update_parameters(iter)
            
            for i in range(self.num_particles):
                r1, r2 = np.random.rand(), np.random.rand()
                self.vel[i] = (self.w * self.vel[i] +
                               self.c1 * r1 * (self.pbest[i] - self.pos[i]) +
                               self.c2 * r2 * (self.gbest - self.pos[i]))
                self.pos[i] += self.vel[i]
                
                # Appliquer les limites
                self.pos[i] = np.clip(self.pos[i], self.bounds[:, 0], self.bounds[:, 1])
                
                # Évaluation de la nouvelle position
                new_val = self.func(self.pos[i])
                
                # Mise à jour du meilleur personnel
                if new_val < self.pbest_val[i]:
                    self.pbest[i] = self.pos[i]
                    self.pbest_val[i] = new_val
                
                # Mutation pour éviter la stagnation
                self.mutate(i)
            
            # Mise à jour du meilleur global
            best_idx = np.argmin(self.pbest_val)
            if self.pbest_val[best_idx] < self.gbest_val:
                self.gbest = self.pbest[best_idx]
                self.gbest_val = self.pbest_val[best_idx]
            
            print(f'Iteration {iter+1}/{self.max_iter}, Best Value: {self.gbest_val}')
        
        return self.gbest, self.gbest_val

# Exemple d'utilisation : Fonction Sphere (minimisation)
def sphere(x):
    return np.sum(x**2)

# Paramètres
bounds = [(-5, 5)] * 10  # 10 dimensions
pso = APSO(func=sphere, dim=10, bounds=bounds, num_particles=50, max_iter=100)
best_pos, best_val = pso.optimize()
print("Best solution:", best_pos, "Best value:", best_val)

Iteration 1/100, Best Value: 15.677112183412461
Iteration 2/100, Best Value: 12.013014943219217
Iteration 3/100, Best Value: 3.7733363489793543
Iteration 4/100, Best Value: 3.7733363489793543
Iteration 5/100, Best Value: 3.6833617843428894
Iteration 6/100, Best Value: 3.3538903870111394
Iteration 7/100, Best Value: 1.7820999910842459
Iteration 8/100, Best Value: 1.1943696975441378
Iteration 9/100, Best Value: 1.1943696975441378
Iteration 10/100, Best Value: 1.1943696975441378
Iteration 11/100, Best Value: 1.1943696975441378
Iteration 12/100, Best Value: 1.1943696975441378
Iteration 13/100, Best Value: 1.1943696975441378
Iteration 14/100, Best Value: 1.0731503675476959
Iteration 15/100, Best Value: 1.0655985938568289
Iteration 16/100, Best Value: 1.0553921694728647
Iteration 17/100, Best Value: 0.6770803362776476
Iteration 18/100, Best Value: 0.6403924212580677
Iteration 19/100, Best Value: 0.6403924212580677
Iteration 20/100, Best Value: 0.6403924212580677
Iteration 21/100, Best Value:

 DFS avec 2𝑚

  (toutes les combinaisons possibles)
 Problème : Sélectionner n'importe quel nombre de sous-ensembles pour couvrir le plus d'éléments possible.


In [ ]:
import itertools

def fitness_function(subset_indices, subsets, universe):
    """Calcule la couverture des éléments par les sous-ensembles sélectionnés."""
    covered_elements = set()
    for idx in subset_indices:
        covered_elements.update(subsets[idx])
    return len(covered_elements)

def dfs_cover(subsets, universe, k, current_selection=[], best_selection=None, best_coverage=0, index=0):
    """Algorithme DFS pour explorer toutes les combinaisons possibles de k sous-ensembles."""
    # Condition d'arrêt : si k sous-ensembles ont été sélectionnés
    if len(current_selection) == k:
        coverage = fitness_function(current_selection, subsets, universe)
        if coverage > best_coverage:
            return list(current_selection), coverage
        return best_selection, best_coverage

    # Condition d'arrêt : si on a exploré tous les sous-ensembles
    if index >= len(subsets):
        return best_selection, best_coverage

    # Cas où on inclut le sous-ensemble actuel
    best_selection, best_coverage = dfs_cover(subsets, universe, k, current_selection + [index], best_selection, best_coverage, index + 1)

    # Cas où on exclut le sous-ensemble actuel
    best_selection, best_coverage = dfs_cover(subsets, universe, k, current_selection, best_selection, best_coverage, index + 1)

    return best_selection, best_coverage


universe = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}  
subsets = [
    {1, 2, 3}, {3, 4, 5}, {5, 6, 7}, {7, 8, 9}, {2, 4, 6, 8}, {1, 5, 9, 10}
]
k = 3  


best_solution, best_coverage = dfs_cover(subsets, universe, k)

print("Meilleure sélection de sous-ensembles :", best_solution)
print("Nombre d'éléments couverts :", best_coverage)


Meilleure sélection de sous-ensembles : [0, 4, 5]
Nombre d'éléments couverts : 9
